In [ ]:
# 必要的导入
import pickle
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import backtrader as bt
import backtrader.analyzers as btanalyzers
import optuna
import time
from datetime import datetime, timedelta
import concurrent.futures
import matplotlib.pyplot as plt
import math
from typing import Dict, List, Tuple, Any, Optional

# 在此处添加全局缓存字典，用于缓存数据加载结果和数据完整性检查结果
_data_feed_cache = {}
_data_completeness_cache = {}

c:\Users\x7498\anaconda3\envs\backtrader\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 添加配置单元格

In [2]:
# 导入 MeanReverter 策略
from MeanReverter import MeanReverter

# 全局配置
CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': MeanReverter,
        'name': MeanReverter.__name__
    },
    
    # 如果 selected_symbols 为空，则通过 get_all_symbols 自动获取所有交易对
    'selected_symbols': [
        "BTCUSDT",
        "ETHUSDT",
        "SOLUSDT",
        "XRPUSDT",
        "DOGEUSDT",
        "BNBUSDT",
        "OMUSDT",
        "ADAUSDT",
        "ENAUSDT",
        "LTCUSDT",
        "SUIUSDT",
        "1000PEPEUSDT",
        "AVAXUSDT",
        "LINKUSDT",
        "AAVEUSDT",
        "TRXUSDT",
        "ONDOUSDT",
        "WLDUSDT",
        "DOTUSDT",
        "APTUSDT",
        "UNIUSDT",
        "FILUSDT",
        "JUPUSDT",
        "1000SHIBUSDT",
        "SEIUSDT",
        "ARBUSDT",
        "ATOMUSDT",
        "LDOUSDT",
        "ALCHUSDT",
        "OPUSDT",
    ],

    'data_path': r'\\znas\Main\futures',
    'start_date': '2024-04-01',
    'end_date': '2025-02-01',
    'source_timeframe': '1m',
    'target_timeframes': ['1h'],  # WalkForward测试的时间周期
    
    # 文件保存配置
    'reports_path': 'walkforward_reports',
    'results_filename_template': 'walkforward_results_{strategy_name}_{start_date}-{end_date}.csv',
    'equity_curves_dir': 'equity_curves',
    
    # 进度保存配置
    'progress_settings': {
        'save_dir': 'walkforward_progress',  # 进度保存目录
        'filename_template': '{symbol}_{timeframe}_period_{period}.pkl',  # 进度文件名模板
        'save_interval': 1,  # 每多少个周期保存一次进度
        'auto_resume': True  # 是否自动从上次进度继续
    },
    
    # 回测参数配置
    'commission': 0.0004,
    'slippage': 0.001,
    'initial_capital': 100000,
    
    # 优化参数范围配置
    'optimization_params': {
        'frequency': range(14, 31, 2),            # 14,16,18,20,22,24,26,28,30
        'rsiFrequency': range(30, 45, 2),         # 30,32,34,36,38,40,42,44
        'buyZoneDistance': range(1, 8, 1),        # 1,2,3,4,5,6,7
        'avgDownATRSum': range(3, 8, 1),          # 3,4,5,6,7
        'useAbsoluteRSIBarrier': [True, False],   # True,False
        'barrierLevel': range(54, 65, 2),         # 54,56,58,60,62,64
        'pyramiding': range(2, 5, 1)              # 2,3,4
    },
    
    
    # WalkForward 特定配置
    'walkforward_settings': {
        'optimization_period_days': 84,   # 优化期长度(天)
        'out_of_sample_period_days': 14,   # 样本外测试期长度(天)
        'n_trials': 188,                    # 每个优化周期的optuna试验次数
        'n_jobs': 30,                       # 并行作业数
        'top_n_params': 5                  # 保存前N个最佳参数结果
    }
}

## 1. 数据加载函数

In [3]:
def get_timeframe_params(timeframe_str):
    """
    将时间周期字符串转换为 backtrader 的 timeframe 和 compression 参数
    """
    if timeframe_str.endswith('min'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('min', '')))
    elif timeframe_str.endswith('h') or timeframe_str.endswith('H'):
        minutes = int(timeframe_str.replace('h', '').replace('H', '')) * 60
        return (bt.TimeFrame.Minutes, minutes)
    elif timeframe_str.endswith('D'):
        return (bt.TimeFrame.Days, 1)
    elif timeframe_str == '1m':
        return (bt.TimeFrame.Minutes, 1)
    else:
        raise ValueError(f"不支持的时间周期格式: {timeframe_str}")
    
def load_and_resample_data(symbol, start_date, end_date, target_timeframe):
    """
    加载并重采样期货数据，并缓存已经重采样后的 DataFrame 以避免重复 I/O 操作
    
    参数:
    - symbol: 交易对
    - start_date: 开始日期时间戳
    - end_date: 结束日期时间戳
    - target_timeframe: 目标时间框架
    
    返回:
    - backtrader 数据馈送对象
    """
    # 转换时间戳为日期字符串
    start_date_str = datetime.fromtimestamp(start_date).strftime('%Y-%m-%d')
    end_date_str = datetime.fromtimestamp(end_date).strftime('%Y-%m-%d')
    
    # 从CONFIG获取数据路径和源时间周期
    data_path = CONFIG['data_path']
    source_timeframe = CONFIG['source_timeframe']
    
    # 构造缓存键
    key = (symbol, start_date_str, end_date_str, source_timeframe, target_timeframe, data_path)
    if key in _data_feed_cache:
        # 如果缓存中有，返回新的数据馈送对象（注意拷贝，防止被修改）
        cached_df = _data_feed_cache[key]
        timeframe, compression = get_timeframe_params(target_timeframe)
        data_feed = bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date_str),
            todate=pd.to_datetime(end_date_str)
        )
        
        # 添加clone方法，这样可以快速创建数据副本而不需要重新执行IO
        data_feed.clone = lambda: bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date_str),
            todate=pd.to_datetime(end_date_str)
        )
        
        return data_feed
    
    # 生成日期范围
    date_range = pd.date_range(start=start_date_str, end=end_date_str, freq='D')
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 顺序读取文件，不使用线程池
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并、排序，以及重采样数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    combined_df.set_index('datetime', inplace=True)
    
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    backtesting_df = backtesting_df.dropna()
    
    # 将结果缓存在全局变量中（使用拷贝，以免后续被修改）
    _data_feed_cache[key] = backtesting_df.copy()
    
    timeframe, compression = get_timeframe_params(target_timeframe)
    data_feed = bt.feeds.PandasData(
        dataname=backtesting_df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date_str),
        todate=pd.to_datetime(end_date_str)
    )
    
    # 添加clone方法
    data_feed.clone = lambda: bt.feeds.PandasData(
        dataname=backtesting_df.copy(),
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date_str),
        todate=pd.to_datetime(end_date_str)
    )
    
    return data_feed

## 2. 回测分析器类

In [4]:
class AcctStats(bt.Analyzer):
    """账户统计分析器"""
    
    def __init__(self):
        self.start_val = self.strategy.broker.get_value()
        self.end_val = None

    def stop(self):
        self.end_val = self.strategy.broker.get_value()

    def get_analysis(self):
        return {"start": self.start_val, "end": self.end_val}

class ValueStats(bt.Analyzer):
    """账户价值跟踪分析器"""
    
    def __init__(self):
        self.val = []

    def next(self): 
        self.val.append(self.strategy.broker.get_value())

    def get_analysis(self):
        return self.val

## 3. 时间框架映射表

In [5]:
# 时间框架到秒数的映射
candle_freq_to_seconds_map = {
    '1Min': 60,
    '3Min': 180,
    '5Min': 300,
    '15Min': 900,
    '1Hour': 3600,
    '4Hour': 14400,
    '6Hour': 21600,
    '1D': 86400
}

## 5. 回测执行函数

In [6]:
def run_backtest(symbol, timeframe, start_timestamp, end_timestamp, strategy_params, capital,
                 plot=False, print_log=False):
    """
    执行单次回测
    """
    try:
        # 加载数据 - 使用clone避免重复加载
        data_feed = load_and_resample_data(symbol, start_timestamp, end_timestamp, timeframe)
        
        # 创建cerebro实例
        cerebro = bt.Cerebro(
            optdatas=True,    
            optreturn=True,   
            runonce=True,     
            preload=True      
        )
        
        # 打印当前运行的策略参数（用于调试）
        if print_log:
            print(f"\n{'-'*20} 回测设置 {'-'*20}")
            print(f"交易对: {symbol} | 时间框架: {timeframe}")
            print(f"时间区间: {datetime.fromtimestamp(start_timestamp).strftime('%Y-%m-%d')} → {datetime.fromtimestamp(end_timestamp).strftime('%Y-%m-%d')}")
        
        # 从interval_params提取最后一组参数(适用于当前测试的参数)
        if 'interval_params' in strategy_params and strategy_params['interval_params']:
            # 获取最后一个时间段的参数
            last_params = strategy_params['interval_params'][-1][1]
            # 添加策略，直接将参数传递给策略，不传递printlog参数
            cerebro.addstrategy(CONFIG['strategy']['class'], **last_params)
            
            # 打印策略参数（用于调试）
            if print_log:
                print(f"\n策略参数:")
                for param_name, param_value in last_params.items():
                    print(f"  - {param_name}: {param_value}")
        else:
            # 没有区间参数时的默认处理
            cerebro.addstrategy(CONFIG['strategy']['class'])
            if print_log:
                print("使用默认策略参数")
        
        # 设置手续费和滑点
        if 'slippage' in strategy_params:
            cerebro.broker.set_slippage_perc(perc=strategy_params['slippage'])
        if 'commission' in strategy_params:
            cerebro.broker.setcommission(commission=strategy_params['commission'])
        
        # 添加分析器
        cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalysis')
        cerebro.addanalyzer(btanalyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, 
                            riskfreerate=0.0, _name='SharpeAnalysis')
        cerebro.addanalyzer(btanalyzers.DrawDown, _name='DrawDownAnalysis')
        cerebro.addanalyzer(AcctStats, _name='ActualAnalysis')
        cerebro.addanalyzer(ValueStats, _name='ValueAnalysis')
        
        # 添加数据和设置初始资金
        cerebro.adddata(data_feed)
        cerebro.broker.setcash(capital)
        
        # 打印起始条件
        if print_log:
            print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')
        
        # 运行回测
        results = cerebro.run()
        
        # 打印最终结果
        if print_log:
            print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')
        
        # 提取分析结果
        result = results[0]
        trade_analysis = result.analyzers.TradeAnalysis.get_analysis()
        sharpe_analysis = result.analyzers.SharpeAnalysis.get_analysis()
        drawdown_analysis = result.analyzers.DrawDownAnalysis.get_analysis()
        returns_analysis = result.analyzers.ActualAnalysis.get_analysis()
        value_analysis = result.analyzers.ValueAnalysis.get_analysis()
        
        # 处理结果
        total_pnl = returns_analysis['end'] - returns_analysis['start']
        num_won_trades = trade_analysis['won']['total'] if 'won' in trade_analysis else 0
        num_lost_trades = trade_analysis['lost']['total'] if 'lost' in trade_analysis else 0
        
        sharpe_ratio = sharpe_analysis['sharperatio'] if 'sharperatio' in sharpe_analysis else 0
        
        win_ratio = num_won_trades / (num_won_trades + num_lost_trades) if (num_won_trades + num_lost_trades) > 0 else 0
        used_capital = capital * 0.5
        percentage_pnl = (total_pnl / used_capital) * 100
        
        max_drawdown = drawdown_analysis['max']['drawdown'] if 'max' in drawdown_analysis else 0
        calmar_ratio = percentage_pnl / math.sqrt((1 + max_drawdown)) if max_drawdown > 0 else 0
        
        # 如果要绘图
        if plot:
            cerebro.plot(style='candle')
            
        return {
            'win_ratio': win_ratio,
            'percentage_pnl': percentage_pnl,
            'sharpe_ratio': sharpe_ratio,
            'calmar_ratio': calmar_ratio,
            'trade_analysis': trade_analysis,
            'value_analysis': value_analysis
        }
        
    except Exception as e:
        print(f"回测执行错误: {str(e)}")
        return {
            'win_ratio': 0.0,
            'percentage_pnl': 0.0,
            'sharpe_ratio': 0.0,
            'calmar_ratio': 0.0,
            'trade_analysis': {},
            'value_analysis': [],
            'error': str(e)
        }

## 6. Optuna优化相关函数

In [7]:
def setup_optuna_optimization_ranges(trial):
    """
    设置Optuna参数搜索空间，使用CONFIG中的范围
    
    参数:
    - trial: Optuna Trial对象
    
    返回:
    - 包含参数值的字典
    """
    params = {}
    
    # 从CONFIG获取参数范围
    opt_params = CONFIG['optimization_params']
    
    # 设置整数参数
    params['frequency'] = trial.suggest_int('frequency', 
                                           opt_params['frequency'][0], 
                                           opt_params['frequency'][1])
    
    params['rsiFrequency'] = trial.suggest_int('rsiFrequency', 
                                              opt_params['rsiFrequency'][0], 
                                              opt_params['rsiFrequency'][1])
    
    # 设置浮点数参数
    params['buyZoneDistance'] = trial.suggest_float('buyZoneDistance', 
                                                   opt_params['buyZoneDistance'][0], 
                                                   opt_params['buyZoneDistance'][1])
    
    params['avgDownATRSum'] = trial.suggest_float('avgDownATRSum', 
                                                 opt_params['avgDownATRSum'][0], 
                                                 opt_params['avgDownATRSum'][1])
    
    # 设置布尔参数
    params['useAbsoluteRSIBarrier'] = trial.suggest_categorical('useAbsoluteRSIBarrier', 
                                                               opt_params['useAbsoluteRSIBarrier'])
    
    # 设置整数参数
    params['barrierLevel'] = trial.suggest_int('barrierLevel', 
                                              opt_params['barrierLevel'][0], 
                                              opt_params['barrierLevel'][1])
    
    params['pyramiding'] = trial.suggest_int('pyramiding', 
                                            opt_params['pyramiding'][0], 
                                            opt_params['pyramiding'][1])
    
    return params

def convert_optuna_params_to_strategy_params(params, end_timestamp):
    """
    转换Optuna优化参数为策略参数
    
    参数:
    - params: Optuna参数
    - end_timestamp: 结束时间戳
    
    返回:
    - 策略参数字典
    """
    strategy_params = {}
    
    # 设置区间参数
    interval_params = [(end_timestamp, params)]
    strategy_params['interval_params'] = interval_params
    
    # 设置其他参数
    strategy_params['commission'] = 0.0005
    strategy_params['slippage'] = 0.001
    
    return strategy_params

def objective_function(trial, symbol, timeframe, start_date, end_date, capital):
    """
    Optuna优化的目标函数
    """
    params = {}
    for param_name, param_range in CONFIG['optimization_params'].items():
        if isinstance(param_range, range):
            params[param_name] = trial.suggest_int(
                name=param_name,
                low=param_range.start,
                high=param_range.stop - 1,  # 减1是因为range的stop是开区间
                step=param_range.step  # 作为关键字参数传递step
            )
        else:
            params[param_name] = trial.suggest_categorical(param_name, param_range)
    
    # 转换参数为策略可用的格式
    strategy_params = convert_optuna_params_to_strategy_params(params, end_date)
    
    # 运行回测
    results = run_backtest(
        symbol, timeframe, start_date, end_date,
        strategy_params, capital, False, False
    )
    
    return results['sharpe_ratio']

In [8]:
def safe_print_results(results):
    """安全打印结果"""
    try:
        # 首先检查结果是否是字典类型
        if not isinstance(results, dict):
            print(f"无效的结果格式: {results}")
            return
            
        # 使用 get 方法安全地获取值，提供默认值
        win_ratio = results.get('win_ratio', 0)
        pnl = results.get('percentage_pnl', 0)
        sharpe = results.get('sharpe_ratio', 0)
        calmar = results.get('calmar_ratio', 0)
        
        # 打印基本指标
        print(f"  胜率: {win_ratio:.2f}")
        print(f"  收益率: {pnl:.2f}%")
        print(f"  Sharpe比率: {sharpe:.4f}")
        print(f"  Calmar比率: {calmar:.4f}")
        
        # 获取交易分析数据
        trade_analysis = results.get('trade_analysis', {})
        if not trade_analysis:
            print("  没有交易记录")
            return
            
        # 安全地获取交易统计
        won_trades = trade_analysis.get('won', {})
        lost_trades = trade_analysis.get('lost', {})
        
        won_count = won_trades.get('total', 0)
        lost_count = lost_trades.get('total', 0)
        total_trades = won_count + lost_count
        
        if total_trades > 0:
            print(f"\n交易统计:")
            print(f"  总交易次数: {total_trades}")
            print(f"  盈利交易: {won_count}")
            print(f"  亏损交易: {lost_count}")
            
            # 获取盈亏数据
            pnl_data = trade_analysis.get('pnl', {}).get('net', {})
            if pnl_data:
                total_pnl = pnl_data.get('total', 0)
                avg_won = won_trades.get('pnl', {}).get('average', 0)
                avg_lost = lost_trades.get('pnl', {}).get('average', 0)
                
                print(f"  总盈亏: {total_pnl:.2f}")
                print(f"  平均盈利: {avg_won:.2f}")
                print(f"  平均亏损: {avg_lost:.2f}")
        else:
            print("  该期间没有完成的交易")
            
    except Exception as e:
        print(f"打印结果时出错: {str(e)}")
        print(f"原始结果: {results}")




## 7. WalkForward核心函数

In [9]:
def get_progress_path(symbol, timeframe, period=None):
    """获取进度文件路径"""
    save_dir = Path(CONFIG['progress_settings']['save_dir'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    if period is None:
        # 返回目录路径
        return save_dir
    else:
        # 返回特定进度文件路径
        filename = CONFIG['progress_settings']['filename_template'].format(
            symbol=symbol,
            timeframe=timeframe,
            period=period
        )
        return save_dir / filename

def save_progress(data, symbol, timeframe, current_period):
    """保存优化进度"""
    try:
        filepath = get_progress_path(symbol, timeframe, current_period)
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        print(f"\n进度已保存: {filepath}")
        
        # 保存一个进度索引文件
        index_file = get_progress_path(symbol, timeframe) / 'progress_index.json'
        index_data = {
            'last_period': current_period,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'symbol': symbol,
            'timeframe': timeframe
        }
        with open(index_file, 'w') as f:
            json.dump(index_data, f, indent=4)
            
    except Exception as e:
        print(f"保存进度时出错: {str(e)}")

def load_progress(symbol, timeframe):
    """加载最新的优化进度"""
    try:
        progress_dir = get_progress_path(symbol, timeframe)
        index_file = progress_dir / 'progress_index.json'
        
        if not index_file.exists():
            print("未找到进度文件，将从头开始优化")
            return None, 1
        
        # 读取索引文件
        with open(index_file, 'r') as f:
            index_data = json.load(f)
        
        last_period = index_data['last_period']
        progress_file = get_progress_path(symbol, timeframe, last_period)
        
        if not progress_file.exists():
            print("进度文件不存在，将从头开始优化")
            return None, 1
        
        # 加载进度数据
        with open(progress_file, 'rb') as f:
            data = pickle.load(f)
        
        print(f"\n已加载进度:")
        print(f"交易对: {symbol}/{timeframe}")
        print(f"上次优化周期: {last_period}")
        print(f"保存时间: {index_data['timestamp']}")
        
        return data, last_period + 1  # 返回下一个要处理的周期
        
    except Exception as e:
        print(f"加载进度时出错: {str(e)}")
        return None, 1

def create_optuna_study(storage=None, study_name=None):
    """
    创建Optuna Study
    
    参数:
    - storage: 存储URI
    - study_name: Study名称
    
    返回:
    - Optuna Study对象
    """
    if storage and study_name:
        return optuna.create_study(
            study_name=study_name,
            storage=storage,
            load_if_exists=True,
            direction="minimize"
        )
    else:
        return optuna.create_study(direction="minimize")


def OptunaParallelWalkForwardAnalysis(symbol, timeframe, start_date, end_date, 
                                       optimization_period, out_of_sample_period,
                                       n_trials, capital, n_jobs=1):
    """
    单交易对的Optuna WalkForward优化
    
    参数:
    - symbol: 交易对
    - timeframe: 时间框架
    - start_date: 开始日期时间戳
    - end_date: 结束日期时间戳
    - optimization_period: 优化期长度(秒)
    - out_of_sample_period: 样本外测试期长度(秒)
    - n_trials: Optuna优化尝试次数
    - capital: 初始资金
    - n_jobs: 并行作业数
    
    返回:
    - 包含每个时间段最优参数和性能的字典
    """
    # 尝试加载之前的优化进度
    saved_data, current_period = load_progress(symbol, timeframe)
    
    if saved_data:
        studies = saved_data.get('studies', {})
        best_params = saved_data.get('best_params', {})
        optimization_start_date = saved_data.get('next_optimization_start', start_date - optimization_period)
        optimization_end_date = saved_data.get('next_optimization_end', start_date)
        testing_start_date = saved_data.get('next_testing_start', start_date)
        testing_end_date = saved_data.get('next_testing_end', start_date + out_of_sample_period)
    else:
        studies = {}
        best_params = {}
        optimization_start_date = start_date - optimization_period
        optimization_end_date = start_date
        testing_start_date = start_date
        testing_end_date = start_date + out_of_sample_period
        current_period = 1

    # 计算总优化区间数
    total_periods = int((end_date - start_date) / out_of_sample_period) + 1

    print(f"\n{'='*60}")
    print(f"开始/继续WalkForward优化: {symbol}/{timeframe}")
    print(f"当前进度: 第{current_period}/{total_periods}个优化区间")
    print(f"{'='*60}\n")
    
    try:
        # 循环进行WalkForward优化
        while optimization_end_date < end_date:
            # 创建时间段的key
            optimization_start_date_key = datetime.fromtimestamp(optimization_start_date).strftime("%Y-%m-%d %H:%M:%S")
            optimization_end_date_key = datetime.fromtimestamp(optimization_end_date).strftime("%Y-%m-%d %H:%M:%S")
            optimization_key = f"{optimization_start_date_key}_{optimization_end_date_key}"
            
            print(f"\n{'-'*60}")
            print(f"优化区间[{current_period}/{total_periods}]: {optimization_start_date_key.split(' ')[0]} → {optimization_end_date_key.split(' ')[0]}")
            print(f"{'-'*60}")
            
            # 创建Optuna Study
            study_name_str = f"{symbol}_{timeframe}_{optimization_key}"
            study = create_optuna_study(study_name=study_name_str)
            
            # 定义优化函数
            def optimize_func(trial):
                return objective_function(
                    trial, symbol, timeframe, 
                    optimization_start_date, optimization_end_date, 
                    capital
                )
            
            # 运行优化
            print(f"开始运行{n_trials}次试验，并行作业数: {n_jobs}")
            start_opt_time = time.time()
            study.optimize(optimize_func, n_trials=n_trials, n_jobs=n_jobs)
            end_opt_time = time.time()
            
            # 保存优化结果
            studies[optimization_key] = study
            best_params = study.best_params
            
            print(f"\n优化完成 [耗时: {(end_opt_time - start_opt_time):.1f}秒]")
            print(f"最佳参数 (Sharpe={-study.best_value:.4f}):")
            for param_name, param_value in best_params.items():
                print(f"  - {param_name}: {param_value}")
                
            # 立即进行样本外测试
            print("\n立即执行样本外测试...")
            oos_strategy_params = {
                'interval_params': [(testing_end_date, best_params)],
                'commission': CONFIG['commission'],
                'slippage': CONFIG['slippage']
            }
            
            oos_results = run_backtest(
                symbol, timeframe, 
                testing_start_date, testing_end_date,
                oos_strategy_params, capital, 
                False, True  # 打印详细日志
            )
            
            # 打印样本外测试结果
            print(f"\n样本外测试结果 ({datetime.fromtimestamp(testing_start_date).strftime('%Y-%m-%d')} → {datetime.fromtimestamp(testing_end_date).strftime('%Y-%m-%d')}):")
            if isinstance(oos_results, dict):
                safe_print_results(oos_results)
            else:
                print(f"无效的样本外测试结果: {oos_results}")
            
            print(f"\n下一个样本外测试区间: {datetime.fromtimestamp(testing_start_date + out_of_sample_period).strftime('%Y-%m-%d')} → {datetime.fromtimestamp(testing_end_date + out_of_sample_period).strftime('%Y-%m-%d')}")
            
            # 保存当前周期进度
            progress_data = {
                'studies': studies,
                'best_params': best_params,
                'next_optimization_start': optimization_start_date + out_of_sample_period,
                'next_optimization_end': optimization_end_date + out_of_sample_period,
                'next_testing_start': testing_start_date + out_of_sample_period,
                'next_testing_end': testing_end_date + out_of_sample_period
            }
            save_progress(progress_data, symbol, timeframe, current_period)
            
            # 更新时间段
            optimization_start_date += out_of_sample_period
            optimization_end_date += out_of_sample_period
            testing_start_date += out_of_sample_period
            testing_end_date += out_of_sample_period
            current_period += 1
    except Exception as e:
        print(f"优化过程中断: {str(e)}")
        raise e

    return {
        'studies': studies,
        'best_params': best_params
    }

## 8. 样本外测试函数

In [10]:
def get_best_params_from_study(study, rank=0):
    """
    从Study中获取指定排名的最佳参数
    
    参数:
    - study: Optuna Study对象
    - rank: 参数排名(0为最佳)
    
    返回:
    - 参数字典
    """
    # 获取所有试验并按目标值排序
    trials = sorted(study.trials, key=lambda t: t.value)
    
    if rank < len(trials):
        return trials[rank].params
    else:
        return study.best_params  # 如果指定排名超出范围，返回最佳参数

def create_interval_params_from_study(optimization_results, out_of_sample_period, which_best=0):
    """
    从Study结果创建区间参数
    
    参数:
    - optimization_results: 优化结果字典
    - out_of_sample_period: 样本外期间长度(秒)
    - which_best: 使用第几好的参数(0为最佳)
    
    返回:
    - 区间参数列表、缓冲开始日期、实际开始日期、结束日期
    """
    studies = optimization_results['studies']
    sorted_keys = sorted(studies.keys())
    
    interval_params = []
    out_of_sample_start_date_with_buffer = 0
    out_of_sample_start_date = 0
    end_date = 0
    
    for optimization_key in sorted_keys:
        study = studies[optimization_key]
        
        # 获取优化结束日期
        optimization_end_date_key = optimization_key.split('_')[1]
        optimization_end_date = time.mktime(time.strptime(optimization_end_date_key, "%Y-%m-%d %H:%M:%S"))
        
        # 计算样本外日期
        out_of_sample_start_date_temp = optimization_end_date
        out_of_sample_end_date = out_of_sample_start_date_temp + out_of_sample_period
        
        # 获取最佳参数
        best_params = get_best_params_from_study(study, which_best)
        
        # 初始化缓冲日期(只在第一次)
        if out_of_sample_start_date_with_buffer == 0:
            # 计算最大回溯期
            max_lookback = max(best_params.get('frequency', 14), best_params.get('rsiFrequency', 14))
            needed_lookback_buffer = max_lookback * candle_freq_to_seconds_map.get(timeframe, 3600)
            
            out_of_sample_start_date_with_buffer = out_of_sample_start_date_temp - needed_lookback_buffer
            out_of_sample_start_date = out_of_sample_start_date_temp
            
        # 更新结束日期
        end_date = out_of_sample_end_date
        
        # 添加到区间参数列表
        interval_params.append((out_of_sample_end_date, best_params))
    
    return [interval_params, out_of_sample_start_date_with_buffer, out_of_sample_start_date, end_date]

def generate_out_of_sample_results(optimization_results, symbol, timeframe, 
                                  out_of_sample_period, capital, top_n=5):
    """生成样本外测试结果"""
    out_of_sample_results = []
    
    print(f"\n{'='*60}")
    print(f"开始样本外测试: {symbol}/{timeframe}")
    print(f"测试前{top_n}个最佳参数组合")
    print(f"{'='*60}\n")
    
    for i in range(top_n):
        print(f"\n{'-'*60}")
        print(f"样本外测试 - 参数排名 #{i+1}/{top_n}")
        print(f"{'-'*60}")
        
        # 从Study中获取区间参数
        [interval_params, start_with_buffer, start_date, end_date] = create_interval_params_from_study(
            optimization_results, out_of_sample_period, i
        )
        
        # 创建时间段标识
        start_buffer_key = datetime.fromtimestamp(start_with_buffer).strftime("%Y-%m-%d %H:%M:%S")
        start_key = datetime.fromtimestamp(start_date).strftime("%Y-%m-%d %H:%M:%S")
        end_key = datetime.fromtimestamp(end_date).strftime("%Y-%m-%d %H:%M:%S")
        print(f"测试期间: {start_key.split(' ')[0]} → {end_key.split(' ')[0]} (缓冲开始: {start_buffer_key.split(' ')[0]})")
        
        # 打印区间参数详情
        print("\n使用的参数区间:")
        for idx, param in enumerate(interval_params):
            end_time = datetime.fromtimestamp(param[0]).strftime("%Y-%m-%d")
            print(f"\n区间{idx+1} (截至 {end_time}):")
            for p_name, p_value in param[1].items():
                print(f"  - {p_name}: {p_value}")
        
        # 准备策略参数
        strategy_params = {
            'interval_params': interval_params,
            'commission': 0.0005,
            'slippage': 0.001
        }
        
        # 执行样本外回测
        print("\n执行样本外回测...")
        oos_result = run_backtest(
            symbol, timeframe, start_with_buffer, end_date,
            strategy_params, capital, False, False
        )
        
        # 打印结果摘要
        print(f"\n样本外测试结果 (排名 #{i+1}):")
        print(f"  胜率: {oos_result['win_ratio']:.2f}")
        print(f"  收益率: {oos_result['percentage_pnl']:.2f}%")
        print(f"  Sharpe比率: {oos_result['sharpe_ratio']:.4f}")
        print(f"  Calmar比率: {oos_result['calmar_ratio']:.4f}")
        
        # 保存结果
        result_key = f"{start_buffer_key}_{end_key}"
        out_of_sample_results.append({
            'rank': i,
            'result_key': result_key,
            'results': oos_result,
            'value_curve': oos_result['value_analysis'],
            'symbol': symbol,
            'timeframe': timeframe
        })
    
    return out_of_sample_results

## 9. 多交易对批处理与结果汇总

In [11]:
from tqdm import tqdm

def run_optuna_walkforward_optimization(symbols, timeframes, start_date, end_date, 
                                        optimization_period, out_of_sample_period,
                                        n_trials, capital, n_jobs=1):
    """
    运行多交易对的Optuna WalkForward优化
    
    参数:
    - symbols: 交易对列表
    - timeframes: 时间框架列表
    - start_date: 整体回测开始日期
    - end_date: 整体回测结束日期
    - optimization_period: 优化期长度(秒)
    - out_of_sample_period: 样本外测试期长度(秒)
    - n_trials: Optuna优化尝试次数
    - capital: 初始资金
    - n_jobs: 并行处理的作业数
    
    返回:
    - 所有交易对的优化结果和样本外测试结果
    """
    # 初始化结果存储
    all_optimization_results = {}
    all_oos_results = {}
    
    # 创建交易对-时间框架组合
    pairs = []
    for symbol in symbols:
        for timeframe in timeframes:
            pairs.append((symbol, timeframe))
    
    total_pairs = len(pairs)
    current_pair = 1
    
    # 打印优化概览
    print(f"\n{'#'*80}")
    print("开始WalkForward优化框架")
    print(f"交易对数量: {len(symbols)} | 时间框架数量: {len(timeframes)} | 总组合数: {total_pairs}")
    print(f"时间范围: {datetime.fromtimestamp(start_date).strftime('%Y-%m-%d')} → {datetime.fromtimestamp(end_date).strftime('%Y-%m-%d')}")
    print(f"优化期长度: {int(optimization_period/86400)}天 | 样本外测试期长度: {int(out_of_sample_period/86400)}天")
    print(f"每个周期尝试次数: {n_trials} | 并行作业数: {n_jobs}")
    print(f"{'#'*80}\n")
    
    # 串行处理每个交易对(可以改为并行)
    for symbol, timeframe in pairs:
        print(f"\n{'#'*80}")
        print(f"处理组合 [{current_pair}/{total_pairs}]: {symbol}/{timeframe}")
        print(f"{'#'*80}")
        
        pair_start_time = time.time()
        
        # 执行WalkForward优化
        opt_results = OptunaParallelWalkForwardAnalysis(
            symbol, timeframe, start_date, end_date,
            optimization_period, out_of_sample_period,
            n_trials, capital, n_jobs
        )
        
        # 保存优化结果
        key = f"{symbol}_{timeframe}"
        all_optimization_results[key] = opt_results
        
        # 生成样本外结果
        oos_results = generate_out_of_sample_results(
            opt_results, symbol, timeframe,
            out_of_sample_period, capital, 5
        )
        
        # 保存样本外结果
        all_oos_results[key] = oos_results
        
        pair_end_time = time.time()
        print(f"\n组合 {symbol}/{timeframe} 处理完成 [耗时: {(pair_end_time - pair_start_time):.1f}秒]")
        
        current_pair += 1
    
    return {
        'optimization_results': all_optimization_results,
        'oos_results': all_oos_results
    }

def process_all_results(all_results):
    """
    处理所有交易对的回测结果
    
    参数:
    - all_results: 所有结果的字典
    
    返回:
    - 处理后的结果DataFrame
    """
    rows = []
    
    # 提取样本外结果
    oos_results = all_results['oos_results']
    
    # 处理每个交易对的结果
    for key, result_list in oos_results.items():
        symbol, timeframe = key.split('_')
        
        for result_data in result_list:
            rank = result_data['rank']
            result = result_data['results']
            
            # 创建结果行
            row = {
                'Symbol': symbol,
                'Target Timeframe': timeframe,
                'Rank': rank + 1,  # 让排名从1开始
                'Win Ratio': result['win_ratio'],
                'PnL%': result['percentage_pnl'],
                'Sharpe': result['sharpe_ratio'],
                'Calmar': result['calmar_ratio'],
                'Score': result['sharpe_ratio'] * (1 + result['percentage_pnl']/100)  # 简单评分
            }
            
            # 添加参数
            if result_data['results']['trade_analysis'] and len(result_list) > 0:
                # 添加使用的第一个区间参数作为示例
                params = result_data.get('interval_params', [])[0][1] if len(result_data.get('interval_params', [])) > 0 else {}
                for param_name, param_value in params.items():
                    row[param_name] = param_value
            
            rows.append(row)
    
    # 创建DataFrame
    results_df = pd.DataFrame(rows)
    
    # 按Score排序
    if 'Score' in results_df.columns:
        results_df = results_df.sort_values(by='Score', ascending=False)
    
    return results_df

def save_equity_curves(all_results, output_dir="equity_curves"):
    """
    保存每个交易对的权益曲线到CSV
    
    参数:
    - all_results: 所有结果的字典
    - output_dir: 输出目录
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 提取样本外结果
    oos_results = all_results['oos_results']
    
    # 保存每个交易对的权益曲线
    for key, result_list in oos_results.items():
        for result_data in result_list:
            symbol = result_data['symbol']
            timeframe = result_data['timeframe']
            rank = result_data['rank']
            
            # 获取权益曲线数据
            equity_data = result_data['value_curve']
            
            # 创建DataFrame
            if equity_data:
                df = pd.DataFrame({
                    'value': equity_data
                })
                
                # 保存到CSV
                filename = f"{output_dir}/{symbol}_{timeframe}_rank{rank+1}.csv"
                df.to_csv(filename)
                print(f"Saved equity curve to {filename}")

def aggregate_results_and_generate_report(all_results, output_csv="walkforward_results.csv", 
                                         output_dir="equity_curves"):
    """
    汇总结果并生成报告
    
    参数:
    - all_results: 所有结果的字典
    - output_csv: 输出CSV文件名
    - output_dir: 权益曲线输出目录
    """
    # 处理所有结果
    results_df = process_all_results(all_results)
    
    # 保存权益曲线
    save_equity_curves(all_results, output_dir)
    
    # 确保报告目录存在
    os.makedirs(os.path.dirname(output_csv) if os.path.dirname(output_csv) else '.', exist_ok=True)
    
    # 保存结果到CSV
    results_df.to_csv(output_csv, index=False)
    print(f"结果已保存到 {output_csv}")
    
    # 打印结果摘要
    print("\n===== WalkForward分析结果 =====\n")
    print(f"分析的交易对总数: {len(all_results['oos_results'])}")
    
    # 显示前10个最佳结果
    top10 = results_df.head(10)
    print("\n前10个最佳参数组合:")
    print(top10[['Symbol', 'Target Timeframe', 'Rank', 'PnL%', 'Sharpe', 'Score']])
    
    return results_df

## 10. 主程序执行示例

In [12]:
def main():
    """主程序执行入口"""
    import time
    from datetime import datetime, timedelta
    
    print(f"\n{'#'*80}")
    print(f"启动MeanReverter WalkForward优化框架")
    print(f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'#'*80}\n")
    
    # 从CONFIG获取设置
    symbols = CONFIG['selected_symbols']
    timeframes = CONFIG['target_timeframes']
    
    # 转换日期
    start_date_str = CONFIG['start_date']
    end_date_str = CONFIG['end_date']
    
    start_date = time.mktime(datetime.strptime(start_date_str, "%Y-%m-%d").timetuple())
    end_date = time.mktime(datetime.strptime(end_date_str, "%Y-%m-%d").timetuple())
    
    # 获取WalkForward设置
    wf_settings = CONFIG['walkforward_settings']
    optimization_period = timedelta(days=wf_settings['optimization_period_days']).total_seconds()
    out_of_sample_period = timedelta(days=wf_settings['out_of_sample_period_days']).total_seconds()
    
    n_trials = wf_settings['n_trials']
    capital = CONFIG['initial_capital']
    n_jobs = wf_settings['n_jobs']
    
    print("开始WalkForward优化...")
    start_time = time.time()
    
    try:
        # 运行优化
        results = run_optuna_walkforward_optimization(
            symbols, timeframes, start_date, end_date,
            optimization_period, out_of_sample_period,
            n_trials, capital, n_jobs
        )
        
        # 生成报告
        results_filename = CONFIG['results_filename_template'].format(
            strategy_name=CONFIG['strategy']['name'],
            start_date=start_date_str.replace('-', ''),
            end_date=end_date_str.replace('-', '')
        )
        
        equity_curves_dir = CONFIG['equity_curves_dir']
        
        # 生成报告并保存结果
        final_results = aggregate_results_and_generate_report(results, results_filename, equity_curves_dir)
    except KeyboardInterrupt:
        print("\n优化被用户中断")
        return None, None
    except Exception as e:
        print(f"\n优化过程出错: {str(e)}")
        return None, None
    
    end_time = time.time()
    
    print(f"\n{'#'*80}")
    print(f"WalkForward优化框架执行完成")
    print(f"开始时间: {datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"结束时间: {datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"总耗时: {(end_time - start_time):.2f}秒 ({(end_time - start_time)/3600:.2f}小时)")
    print(f"{'#'*80}\n")
    
    return results, final_results


In [13]:
if __name__ == "__main__":
    results, final_results = main()

[I 2025-03-16 20:41:35,275] A new study created in memory with name: no-name-53660684-bdee-42ec-95bf-5687563c6feb



################################################################################
启动MeanReverter WalkForward优化框架
当前时间: 2025-03-16 20:41:35
################################################################################

开始WalkForward优化...

################################################################################
开始WalkForward优化框架
交易对数量: 30 | 时间框架数量: 1 | 总组合数: 30
时间范围: 2024-04-01 → 2025-02-01
优化期长度: 84天 | 样本外测试期长度: 14天
每个周期尝试次数: 188 | 并行作业数: 30
################################################################################


################################################################################
处理组合 [1/30]: BTCUSDT/1h
################################################################################
未找到进度文件，将从头开始优化

开始/继续WalkForward优化: BTCUSDT/1h
当前进度: 第1/22个优化区间


------------------------------------------------------------
优化区间[1/22]: 2024-01-08 → 2024-04-01
------------------------------------------------------------
开始运行188次试验，并行作业数: 30


[I 2025-03-16 20:41:48,166] Trial 21 finished with value: 0.08235278484602257 and parameters: {'frequency': 30, 'rsiFrequency': 30, 'buyZoneDistance': 1, 'avgDownATRSum': 4, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 64, 'pyramiding': 2}. Best is trial 21 with value: 0.08235278484602257.
[I 2025-03-16 20:41:48,681] Trial 0 finished with value: -0.03546532866778387 and parameters: {'frequency': 20, 'rsiFrequency': 44, 'buyZoneDistance': 5, 'avgDownATRSum': 5, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 62, 'pyramiding': 4}. Best is trial 0 with value: -0.03546532866778387.
[I 2025-03-16 20:41:48,983] Trial 23 finished with value: 0.02981650967453789 and parameters: {'frequency': 28, 'rsiFrequency': 42, 'buyZoneDistance': 5, 'avgDownATRSum': 7, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 64, 'pyramiding': 2}. Best is trial 0 with value: -0.03546532866778387.
[I 2025-03-16 20:41:53,262] Trial 30 finished with value: 0.04987658690888434 and parameters: {'frequency': 22, 'rsiFre


优化完成 [耗时: 109.5秒]
最佳参数 (Sharpe=0.0671):
  - frequency: 20
  - rsiFrequency: 44
  - buyZoneDistance: 2
  - avgDownATRSum: 6
  - useAbsoluteRSIBarrier: False
  - barrierLevel: 64
  - pyramiding: 4

立即执行样本外测试...


[I 2025-03-16 20:43:25,447] A new study created in memory with name: no-name-1fbe4f5a-775d-4852-86a7-5f9024e412c5


...

[I 2025-03-16 20:47:21,009] Trial 3 finished with value: -0.01046924808403314 and parameters: {'frequency': 18, 'rsiFrequency': 34, 'buyZoneDistance': 1, 'avgDownATRSum': 4, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 62, 'pyramiding': 3}. Best is trial 3 with value: -0.01046924808403314.
[I 2025-03-16 20:47:21,097] Trial 1 finished with value: 0.02014459662174926 and parameters: {'frequency': 20, 'rsiFrequency': 42, 'buyZoneDistance': 1, 'avgDownATRSum': 4, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 62, 'pyramiding': 2}. Best is trial 3 with value: -0.01046924808403314.
[I 2025-03-16 20:47:21,669] Trial 2 finished with value: -0.06093277868893655 and parameters: {'frequency': 22, 'rsiFrequency': 34, 'buyZoneDistance': 2, 'avgDownATRSum': 4, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 60, 'pyramiding': 4}. Best is trial 2 with value: -0.06093277868893655.
[I 2025-03-16 20:47:21,909] Trial 0 finished with value: -0.004607471005961259 and parameters: {'frequency': 20, 'rsiFreq


优化完成 [耗时: 117.0秒]
最佳参数 (Sharpe=0.1170):
  - frequency: 28
  - rsiFrequency: 34
  - buyZoneDistance: 3
  - avgDownATRSum: 3
  - useAbsoluteRSIBarrier: False
  - barrierLevel: 56
  - pyramiding: 4

立即执行样本外测试...


[I 2025-03-16 20:49:05,391] A new study created in memory with name: no-name-50be5cf3-1b7b-4b77-bedf-69a973a199de



-------------------- 回测设置 --------------------
交易对: BTCUSDT | 时间框架: 1h
时间区间: 2024-05-13 → 2024-05-27

策略参数:
  - frequency: 28
  - rsiFrequency: 34
  - buyZoneDistance: 3
  - avgDownATRSum: 3
  - useAbsoluteRSIBarrier: False
  - barrierLevel: 56
  - pyramiding: 4
Starting Portfolio Value: 100000.00
Final Portfolio Value: 100294.23

样本外测试结果 (2024-05-13 → 2024-05-27):
打印结果时出错: 'str' object has no attribute 'get'
打印结果时出错: 'str' object has no attribute 'get'
打印结果时出错: 'str' object has no attribute 'get'
打印结果时出错: 'str' object has no attribute 'get'

交易统计:
  总交易次数: 6
  盈利交易: 5
  亏损交易: 1
  总盈亏: 403.87
  平均盈利: 173.02
  平均亏损: -461.25

下一个样本外测试区间: 2024-05-27 → 2024-06-10

进度已保存: walkforward_progress\BTCUSDT_1h_period_4.pkl

------------------------------------------------------------
优化区间[5/22]: 2024-03-04 → 2024-05-27
------------------------------------------------------------
开始运行188次试验，并行作业数: 30


[I 2025-03-16 20:49:18,970] Trial 1 finished with value: 0.012194698307748213 and parameters: {'frequency': 16, 'rsiFrequency': 30, 'buyZoneDistance': 7, 'avgDownATRSum': 4, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 64, 'pyramiding': 4}. Best is trial 1 with value: 0.012194698307748213.
[I 2025-03-16 20:49:19,020] Trial 2 finished with value: -0.048979296126144975 and parameters: {'frequency': 18, 'rsiFrequency': 38, 'buyZoneDistance': 4, 'avgDownATRSum': 3, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 56, 'pyramiding': 3}. Best is trial 2 with value: -0.048979296126144975.
[I 2025-03-16 20:49:20,088] Trial 3 finished with value: 0.0073359724429111345 and parameters: {'frequency': 22, 'rsiFrequency': 40, 'buyZoneDistance': 3, 'avgDownATRSum': 5, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 64, 'pyramiding': 3}. Best is trial 2 with value: -0.048979296126144975.
[I 2025-03-16 20:49:20,921] Trial 4 finished with value: -0.021137082089721877 and parameters: {'frequency': 22, 'rs